# Diversity analysis of microbial communities

In [2]:
import os
from qiime2 import Visualization

data_dir = '1-Denoise'
results_dir = '2-Diversity'

if not os.path.isdir(data_dir):
    os.makedirs(data_dir)

if not os.path.isdir(results_dir):
    os.makedirs(results_dir)

### Phylogeny

In [6]:
! qiime alignment mafft \
    --i-sequences $data_dir/rep-sequences.qza \
    --o-alignment $results_dir/aligned-rep-seqs.qza

! qiime alignment mask \
    --i-alignment $results_dir/aligned-rep-seqs.qza \
    --o-masked-alignment $results_dir/masked-aligned-rep-seqs.qza

! qiime phylogeny fasttree \
    --i-alignment $results_dir/masked-aligned-rep-seqs.qza \
    --o-tree $results_dir/fasttree-tree.qza

! qiime phylogeny midpoint-root \
    --i-tree $results_dir/fasttree-tree.qza \
    --o-rooted-tree $results_dir/fasttree-tree-rooted.qza

Saved FeatureData[AlignedSequence] to: 2-Diversity/aligned-rep-seqs.qza
Saved FeatureData[AlignedSequence] to: 2-Diversity/masked-aligned-rep-seqs.qza
Saved Phylogeny[Unrooted] to: 2-Diversity/fasttree-tree.qza
Saved Phylogeny[Rooted] to: 2-Diversity/fasttree-tree-rooted.qza


### Rarefaction curve

In [8]:
!qiime metadata tabulate \
    --m-input-file 1-Denoise/dada2-stats.qza \
    --o-visualization 1-Denoise/dada2-stats.qzv 

Saved Visualization to: 1-Denoise/dada2-stats.qzv


In [9]:
Visualization.load('1-Denoise/dada2-stats.qzv')

<visualization: Visualization uuid: ba6b26d8-63ec-4e14-8d12-18d8c1e4eeb9>

In [12]:
!qiime feature-table summarize \
    --i-table 1-Denoise/feature-table.qza \
    --m-sample-metadata-file RawData/metadata.tsv \
    --o-visualization 1-Denoise/feature-table.qzv

Saved Visualization to: 1-Denoise/feature-table.qzv


In [15]:
Visualization.load('1-Denoise/feature-table.qzv')

<visualization: Visualization uuid: 47a63b9c-1f95-4757-93fa-adc494a6b0ad>

In [2]:
!qiime diversity alpha-rarefaction --help

Usage: qiime diversity alpha-rarefaction [OPTIONS]

  Generate interactive alpha rarefaction curves by computing rarefactions
  between `min_depth` and `max_depth`. The number of intermediate depths to
  compute is controlled by the `steps` parameter, with n `iterations` being
  computed at each rarefaction depth. If sample metadata is provided,
  samples may be grouped based on distinct values within a metadata column.

Inputs:
  --i-table ARTIFACT FeatureTable[Frequency]
                          Feature table to compute rarefaction curves from.
                                                                    [required]
  --i-phylogeny ARTIFACT  Optional phylogeny for phylogenetic metrics.
    Phylogeny[Rooted]                                               [optional]
Parameters:
  --p-max-depth INTEGER   The maximum rarefaction depth. Must be greater than
    Range(1, None)        min-depth.                                [required]
  --p-metrics TEXT... Choices('berger_parker_d',

In [4]:
!qiime diversity alpha-rarefaction \
    --i-table 1-Denoise/feature-table.qza \
    --i-phylogeny 2-Diversity/fasttree-tree-rooted.qza \
    --p-max-depth 20000 \
    --m-metadata-file RawData/metadata.tsv \
    --p-steps 20 \
    --o-visualization 2-Diversity/rarefaction.qzv

Saved Visualization to: 2-Diversity/rarefaction.qzv


In [17]:
Visualization.load('rarefaction.qzv')

<visualization: Visualization uuid: 86ecd7f7-0c78-4b13-8d39-089f93d16b2e>

### Diversity metrics

In [19]:
!qiime diversity core-metrics-phylogenetic --help

Usage: qiime diversity core-metrics-phylogenetic [OPTIONS]

  Applies a collection of diversity metrics (both phylogenetic and non-
  phylogenetic) to a feature table.

Inputs:
  --i-table ARTIFACT FeatureTable[Frequency]
                          The feature table containing the samples over which
                          diversity metrics should be computed.     [required]
  --i-phylogeny ARTIFACT  Phylogenetic tree containing tip identifiers that
    Phylogeny[Rooted]     correspond to the feature identifiers in the table.
                          This tree can contain tip ids that are not present
                          in the table, but all feature ids in the table must
                          be present in this tree.                  [required]
Parameters:
  --p-sampling-depth INTEGER
    Range(1, None)        The total frequency that each sample should be
                          rarefied to prior to computing diversity metrics.
                                           

In [20]:
!qiime diversity core-metrics-phylogenetic \
    --i-table 1-Denoise/feature-table.qza \
    --i-phylogeny 2-Diversity/fasttree-tree-rooted.qza \
    --p-sampling-depth 10000 \
    --m-metadata-file RawData/metadata.tsv \
    --output-dir 2-Diversity/core-metrics-phylogenetic

Saved FeatureTable[Frequency] to: 2-Diversity/core-metrics-phylogenetic/rarefied_table.qza
Saved SampleData[AlphaDiversity] to: 2-Diversity/core-metrics-phylogenetic/faith_pd_vector.qza
Saved SampleData[AlphaDiversity] to: 2-Diversity/core-metrics-phylogenetic/observed_features_vector.qza
Saved SampleData[AlphaDiversity] to: 2-Diversity/core-metrics-phylogenetic/shannon_vector.qza
Saved SampleData[AlphaDiversity] to: 2-Diversity/core-metrics-phylogenetic/evenness_vector.qza
Saved DistanceMatrix to: 2-Diversity/core-metrics-phylogenetic/unweighted_unifrac_distance_matrix.qza
Saved DistanceMatrix to: 2-Diversity/core-metrics-phylogenetic/weighted_unifrac_distance_matrix.qza
Saved DistanceMatrix to: 2-Diversity/core-metrics-phylogenetic/jaccard_distance_matrix.qza
Saved DistanceMatrix to: 2-Diversity/core-metrics-phylogenetic/bray_curtis_distance_matrix.qza
Saved PCoAResults to: 2-Diversity/core-metrics-phylogenetic/unweighted_unifrac_pcoa_results.qza
Saved PCoAResults to: 2-Diversity/cor

### Group difference

#### 1. Alpha diversity

In [28]:
if not os.path.isdir('2-Diversity/alpha-group-significance'):
    os.mkdir('2-Diversity/alpha-group-significance')

In [27]:
!qiime diversity alpha-group-significance \
    --i-alpha-diversity 2-Diversity/core-metrics-phylogenetic/faith_pd_vector.qza \
    --m-metadata-file RawData/metadata.tsv \
    --o-visualization 2-Diversity/alpha-group-significance/faith_pd_group_significane.qzv

Saved Visualization to: 2-Diversity/alpha-group-significance/faith_pd_group_significane.qzv


In [29]:
Visualization.load('2-Diversity/alpha-group-significance/faith_pd_group_significane.qzv')

<visualization: Visualization uuid: 3053b74f-1cf9-4252-b66f-6eaa8bc95eaf>

#### 2. Beta diversity

In [3]:
Visualization.load('2-Diversity/core-metrics-phylogenetic/bray_curtis_emperor.qzv')

<visualization: Visualization uuid: 1de54cbb-d759-4e3a-b3a0-38aa4fc6957a>